In [61]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import pickle

In [62]:
train = np.array(pd.read_csv('../datasets/Training.csv'))
test = np.array(pd.read_csv('../datasets/Testing.csv'))

In [63]:
symptoms = np.array(pd.read_csv('../datasets/Training.csv').columns.values)[:-1]
for i in range(len(symptoms)):
    symptoms[i] = symptoms[i].replace('_', ' ')

diseases = np.unique(np.array(pd.read_csv('../datasets/Training.csv'))[:, -1])
diseases

with open('SymptomsAndDiseasesList.pickle', 'wb') as f:
    pickle.dump([symptoms, diseases], f)

In [67]:
train_x = train[:, :-1]
train_y = train[:, -1]
test_x = test[:, :-1]
test_y = test[:, -1]
input_dim = train_x.shape[1]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.fit_transform(test_y)

one_hot = OneHotEncoder(sparse=False)

train_y = one_hot.fit_transform(train_y.reshape((len(train),1)))
test_y = one_hot.fit_transform(test_y.reshape((len(test),1)))

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4920, 132) (4920,) (41, 132) (41,)
(4920, 132) (4920, 41) (41, 132) (41, 41)


In [68]:
model = Sequential()

model.add(Dense(170, input_dim = input_dim, activation='tanh'))
model.add(Dense(200, activation='tanh'))
model.add(Dense(140, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(41, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 170)               22610     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               34200     
_________________________________________________________________
dense_3 (Dense)              (None, 140)               28140     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               14100     
_________________________________________________________________
dense_5 (Dense)              (None, 41)                4141      
Total params: 103,191
Trainable params: 103,191
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=50, batch_size=64)
scores = model.evaluate(test_x, test_y)

Epoch 1/50
4920/4920 [==============================] - 5s 975us/step - loss: 0.8407 - acc: 0.9179
Epoch 2/50
4920/4920 [==============================] - 0s 75us/step - loss: 0.0247 - acc: 1.0000
Epoch 3/50
4920/4920 [==============================] - 0s 77us/step - loss: 0.0109 - acc: 1.0000
Epoch 4/50
4920/4920 [==============================] - 0s 78us/step - loss: 0.0064 - acc: 1.0000
Epoch 5/50
4920/4920 [==============================] - 0s 78us/step - loss: 0.0043 - acc: 1.0000: 0s - loss: 0.0048 - acc: 1.
Epoch 6/50
4920/4920 [==============================] - 0s 77us/step - loss: 0.0032 - acc: 1.0000
Epoch 7/50
4920/4920 [==============================] - 0s 77us/step - loss: 0.0024 - acc: 1.0000
Epoch 8/50
4920/4920 [==============================] - 0s 75us/step - loss: 0.0019 - acc: 1.0000
Epoch 9/50
4920/4920 [==============================] - 0s 75us/step - loss: 0.0016 - acc: 1.0000
Epoch 10/50
4920/4920 [==============================] - 0s 76us/step - loss: 0.0013 - a

In [73]:
print(model.metrics_names[1], "=", scores[1] * 100)

acc = 100.0


In [74]:
model.save("DiseasePredictor.h5")